In [1]:
import bokeh as bk
import bokeh.models as models
import bokeh.plotting
from bokeh.plotting import figure, show
import pandas as pd
import polars as pl

In [2]:
df = pd.DataFrame()
df['x'] = [1,2,3,4,5]
df['y'] = [5,4,3,2,1]

In [3]:
source = models.sources.ColumnDataSource(df)

In [4]:
fig = figure()
fig.scatter(source=source)

GlyphRenderer(id='p1038', ...)

In [29]:
show(fig)

In [7]:
pl.DataFrame(df)

x,y
i64,i64
1,5
2,4
3,3
4,2
5,1


In [44]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Legend

# Sample data
source = ColumnDataSource(data=dict(
    x=[1, 2, 3, 4, 5],
    y=[6, 7, 2, 4, 5],
    x2=[1, 2, 3, 4, 5],
    y2=[5, 6, 3, 2, 4]
))

# Create a new plot
p = figure()

# Add glyphs with a legend label
line = p.line('x', 'y', source=source, legend_label="Line", line_width=2)
circle = p.circle('x2', 'y2', source=source, size=20, color="navy", legend_label="Circle")

# Manually create a legend (optional, if more control is needed)
legend = Legend(items=[
    ("Line", [line]),
    ("Circle", [circle])
], location="center")

# Clicking the legend's items will hide/mute the glyph
p.legend.click_policy = "hide"  # Can also be "mute"

# Add the legend manually if created (optional)
# p.add_layout(legend)

# Show the result
show(p)


In [ ]:
import os
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io.export import get_svg, export_svg
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF
from reportlab.graphics.shapes import Drawing
from figs._fig import Fig
import plotly.io as pio
import xml.etree.ElementTree as ETree
import svgpathtools as svgtools

class BokehScalableFigure:
    def __init__(
            self,
            bokeh_figure: figure=None,
            plot_width=17, 
            plot_height=11, 
            x_scale=0.25, 
            y_scale=0.25, 
            x_start=None,
            y_start=None
    ):
        # Basic plot setup
        self.dot_per_inch = 72 #  used to convert plot dimensions to inches, 72 is the standard default
        self.plot_width = plot_width * self.dot_per_inch
        self.plot_height = plot_height * self.dot_per_inch
        self.x_scale = x_scale  # real-world units per inch
        self.y_scale = y_scale  # real-world units per inch
        self.x_start = x_start  # minimum x value in real-world units
        self.y_start = y_start  # minimum y value in real-world units
        if bokeh_figure is None:
            self.figure = figure(
                width=self.plot_width, height=self.plot_height,
                output_backend="svg")
        else:
            self.figure = bokeh_figure
            self.figure.width = self.plot_width
            self.figure.height = self.plot_height
            self.figure.output_backend='svg'

        # Default file paths
        self.svg_path = 'temp_plot.svg'
        self.pdf_path = 'temp_plot.pdf'

    def add_line(self, x, y, **kwargs):
        self.figure.line(x, y, **kwargs)

    def write_svg(self):
        # Save the current figure as svg
        export_svg(self.figure, filename=self.svg_path)
    
    @property
    def x_scale(self):
        if self.x_scale is None:
            self.x_scale = 1
        return self.x_scale
    
    @x_scale.setter
    def x_scale(self, val):
        self.x_scale = val
        
    @property
    def x_start(self):
        if self.x_start is None:
            self.x_start = 0
        return self.x_start
    
    @x_start.setter
    def x_start(self, val):
        self.x_start = val
        
    @property
    def y_scale(self):
        if self.y_scale is None:
            self.y_scale = 1 
        return self.y_scale
    
    @y_scale.setter
    def y_scale(self, val):
        self.y_scale = val
        
    @property
    def y_start(self):
        if self.y_start is None:
            self.y_start = 0 
        return self.y_start
    
    @y_start.setter
    def y_start(self, val):
        self.y_start = val
    
    @property   
    def root(self):
        root = ETree.parse(self.svg_path).getroot()
        self.root = root
        return self.root
        
    def get_scaled_grid_dimensions(self):
        """Parses the figure SVG to get the scaled x and y ranges based on the starting.
        Determines the actual axes spans in inches and scales the ranges so that the
        specified x_scale and y_scale are true."""
        root = self.root
        elements = [element for element in root.iter()]
        #  get the bounding box of the grid of the svg plot exported from Bokeh. the element with the
        #  index of 5 encompases the grid area of the plot, so we get the bbox from this element
        bbox = svgtools.parse_path(elements[5].attrib['d']).bbox()
        x_dot_min, y_dot_min = bbox[0], bbox[2]
        x_dot_span, y_dot_span = bbox[1] - bbox[0], bbox[3] - bbox[2]
        x_length_grid_in_inches = x_dot_span / self.dot_per_inch
        y_height_grid_in_inches = y_dot_span / self.dot_per_inch
        
        x_unit_span = x_length_grid_in_inches * self.x_scale
        y_unit_span = y_height_grid_in_inches * self.y_scale
        x_unit_min, y_unit_min = self.x_start, self.y_start
        
        x_unit_max = x_unit_min + x_unit_span
        y_unit_max = y_unit_min + y_unit_span
        x_range, y_range = (x_unit_min, x_unit_max), (y_unit_min, y_unit_max)
        
        return x_range, y_range
        

    def write_scaled_pdf(self):
        svg_path = self.svg_path
        pdf_path = self.pdf_path
        
        #  write and parse inital svg, adust the figure ranges, and then write the scaled svg
        self.write_svg()
        print('wrote 1st svg')
        x_range, y_range = self.get_scaled_grid_dimensions()
        self.figure.x_range = Range1d(x_range)
        self.figure.y_range = Range1d(y_range)
        self.write_svg()
        print('wrote scaled svg')
                
        # Convert SVG to PDF while maintaining scale
        drawing = svg2rlg(svg_path)

        # Create a ReportLab drawing and render the SVG drawing into it
        rendered_drawing = Drawing(drawing.width, drawing.height)
        rendered_drawing.add(drawing)

        # Write to a PDF
        renderPDF.drawToFile(rendered_drawing, pdf_path)
        print(f'PDF exported to {pdf_path}')
    

# Usage
if __name__ == "__main__":
    bokeh_fig = BokehScalableFigure()
    bokeh_fig.add_line(x=[1, 2, 3, 4], y=[4, 3, 5, 2], line_width=2, color="blue", legend_label='ground line')
    bokeh_fig.write_scaled_pdf()  # Convert and scale to PDF

In [3]:
root = ETree.parse('temp_plot.svg').getroot()
elements = [element for element in root.iter()]
#  get the bounding box of the grid of the svg plot exported from Bokeh
bbox = svgtools.parse_path(elements[5].attrib['d']).bbox()
x_min, y_min = bbox[0], bbox[2]
x_length_grid_in_inches = (bbox[1] - bbox[0]) / 72
y_height_grid_in_inches = (bbox[3] - bbox[2]) / 72

In [4]:
x_length_grid_in_inches

16.125